In [7]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from utils import load_resources, clean_text, insert_prediction_history  
# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load the data
data = pd.read_csv('Suicide_Detection.csv')  # Adjust the path if necessary

# Function to clean text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)  # Remove non-word characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'\b\w{1,2}\b', '', text)  # Remove short words
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text
data['clean_text'] = data['text'].apply(clean_text)

# Split the data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['clean_text'], data['class'], test_size=0.2, random_state=42)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\praba\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
pip install tensorflow


  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.4.0-cp312-cp312-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ----------------------- -------------- 30.7/48.7 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 48.7/48.7 kB 492.9 kB/s eta 0:00:00
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/385.2 M

In [28]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [38]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [40]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [42]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalMaxPooling1D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
import pickle

print("TensorFlow version:", tf.__version__)
print("All libraries are correctly installed.")


TensorFlow version: 2.17.0
All libraries are correctly installed.


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['clean_text'], data['class'], test_size=0.2, random_state=42)

# Encode the labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Verify the data types of the labels
print(y_train.dtype)  # Should be int
print(y_test.dtype)   # Should be int


int32
int32


In [4]:
data.head()

,Unnamed: 0,text,class,clean_text
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide,ex wife threatening suiciderecently left wife ...
1,3,Am I weird I don't get affected by compliments...,non-suicide,weird get affected compliments coming someone ...
2,4,Finally 2020 is almost over... So I can never ...,non-suicide,finally 2020 almost never hear 2020 bad year e...
3,8,i need helpjust help me im crying so hard,suicide,need helpjust help im crying hard
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide,losthello name adam 16 struggling years afraid...


In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences
max_len = 50
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)


In [9]:
import numpy as np

# Load GloVe embeddings
embeddings_index = {}
with open('glove.6B.300d.txt', encoding='utf-8') as f:  # Ensure the file is in the correct directory
    for line in f:
        values = line.split()
        word = values[0]
        embedding_vector = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding_vector

embedding_dim = 300
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [20]:
import numpy as np

# Load GloVe embeddings
embeddings_index = {}
with open('glove.6B.300d.txt', encoding='utf-8') as f:  # Ensure the file is in the correct directory
    for line in f:
        values = line.split()
        word = values[0]
        embedding_vector = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding_vector

embedding_dim = 300
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalMaxPooling1D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, embedding_dim, weights=[embedding_matrix], trainable=False))
model.add(LSTM(128, return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=SGD(learning_rate=0.1, momentum=0.9), metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(X_train_pad, y_train, epochs=20, batch_size=256, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/20
581/581 ━━━━━━━━━━━━━━━━━━━━ 101s 166ms/step - accuracy: 0.8540 - loss: 0.3320 - val_accuracy: 0.9115 - val_loss: 0.2269
Epoch 2/20
581/581 ━━━━━━━━━━━━━━━━━━━━ 147s 175ms/step - accuracy: 0.9150 - loss: 0.2185 - val_accuracy: 0.9229 - val_loss: 0.2021
Epoch 3/20
581/581 ━━━━━━━━━━━━━━━━━━━━ 95s 164ms/step - accuracy: 0.9266 - loss: 0.1907 - val_accuracy: 0.9230 - val_loss: 0.1971
Epoch 4/20
581/581 ━━━━━━━━━━━━━━━━━━━━ 142s 163ms/step - accuracy: 0.9327 - loss: 0.1753 - val_accuracy: 0.9266 - val_loss: 0.1900
Epoch 5/20
581/581 ━━━━━━━━━━━━━━━━━━━━ 98s 169ms/step - accuracy: 0.9415 - loss: 0.1573 - val_accuracy: 0.9304 - val_loss: 0.1822
Epoch 6/20
581/581 ━━━━━━━━━━━━━━━━━━━━ 95s 163ms/step - accuracy: 0.9467 - loss: 0.1437 - val_accuracy: 0.9290 - val_loss: 0.1877
Epoch 7/20
581/581 ━━━━━━━━━━━━━━━━━━━━ 140s 160ms/step - accuracy: 0.9509 - loss: 0.1320 - val_accuracy: 0.9300 - val_loss: 0.1832
Epoch 8/20
581/581 ━━━━━━━━━━━━━━━━━━━━ 142s 160ms/step - accuracy: 0.9551 - lo

In [24]:
from sklearn.metrics import classification_report

y_pred = (model.predict(X_test_pad) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))


1451/1451 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step
              precision    recall  f1-score   support

           0       0.92      0.94      0.93     23287
           1       0.94      0.92      0.93     23128

    accuracy                           0.93     46415
   macro avg       0.93      0.93      0.93     46415
weighted avg       0.93      0.93      0.93     46415



In [30]:
import pickle

model.save('suicide_detection_model.h5')
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [32]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# Load the trained model and tokenizer
model = load_model('suicide_detection_model.h5')
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Sample test texts
test_texts = [
    "I can't find any reason to keep going. Every day feels like a struggle, and I feel so alone and worthless.",
    "I had such a wonderful day today! I spent the afternoon with my family at the park, and we had a picnic together.",
    "I'm feeling very depressed and I don't know how to cope with this pain anymore.",
    "I am looking forward to starting my new job next week. It's an exciting opportunity for me."
]

# Function to clean text (ensure it matches the preprocessing done during training)
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)  # Remove non-word characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'\b\w{1,2}\b', '', text)  # Remove short words
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Clean and tokenize the test texts
test_texts_cleaned = [clean_text(text) for text in test_texts]
test_sequences = tokenizer.texts_to_sequences(test_texts_cleaned)
test_padded = pad_sequences(test_sequences, maxlen=50)

# Predict using the trained model
predictions = model.predict(test_padded)

# Display the results
for text, prediction in zip(test_texts, predictions):
    print(f"Text: {text}")
    print(f"Prediction: {prediction[0] * 100:.2f}% Suicidal\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Text: I can't find any reason to keep going. Every day feels like a struggle, and I feel so alone and worthless.
Prediction: 97.45% Suicidal

Text: I had such a wonderful day today! I spent the afternoon with my family at the park, and we had a picnic together.
Prediction: 0.76% Suicidal

Text: I'm feeling very depressed and I don't know how to cope with this pain anymore.
Prediction: 97.61% Suicidal

Text: I am looking forward to starting my new job next week. It's an exciting opportunity for me.
Prediction: 16.91% Suicidal

